In [12]:
from src.capstone.evaluation import Evaluator
from src.capstone.cbf import CBF
from src.capstone.settings import Env

from double_integrator import DoubleIntegratorEnv
import gymnasium as gym

from bound_propagation.polynomial import Pow
from bound_propagation.linear import FixedLinear

import numpy as np

import torch
import torch.nn as nn

import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'src'

$$B = \begin{bmatrix}
\frac{1}{2}\Delta t^2 & 0\\
0 & \frac{1}{2}\Delta t^2\\
\Delta t & 0\\
0 & \Delta t
\end{bmatrix}$$

$$Q = B B^T = \begin{bmatrix}
\frac{1}{2}\Delta t^2 & 0\\
0 & \frac{1}{2}\Delta t^2\\
\Delta t & 0\\
0 & \Delta t
\end{bmatrix} 

\begin{bmatrix}
\frac{1}{2}\Delta t^2 & 0 & \Delta t & 0\\
0 & \frac{1}{2}\Delta t^2 & 0 & \Delta t
\end{bmatrix} 

= \begin{bmatrix}
\frac{1}{4}\Delta t^4 & 0 & \frac{1}{2}\Delta t^3 & 0\\
0 & \frac{1}{4}\Delta t^4 & 0 & \frac{1}{2}\Delta t^3\\
\frac{1}{2}\Delta t^3 & 0 & \Delta t^2 & 0\\
0 & \frac{1}{2}\Delta t^3 & 0 & \Delta t^2
\end{bmatrix}$$

In [ ]:
class NNDM(nn.Sequential):
    # input [x pos, y pos, x speed, y speed, ux, uy]
    
    def __init__(self):
        super(NNDM, self).__init__()
        self.dt = 0.05
        
        layer = nn.Linear(6, 4, bias=False)
        weights = torch.tensor([[1., 0., self.dt, 0., ((self.dt**2)/2), 0], 
                                [0., 1., 0., self.dt, 0., ((self.dt**2)/2)], 
                                [0., 0., 1., 0., self.dt, 0.], 
                                [0., 0., 0., 1., 0., self.dt]])
        layer.weight = nn.Parameter(weights)
        
        super(NNDM, self).__init__(layer)

nndm = NNDM()

In [ ]:
class Agent(nn.Module):
    def __init__(self):
        super(Agent, self).__init__()
        
        self.layer = nn.Linear(2, 2)
        
        # dummy output of u=0
        self.layer.weight = nn.Parameter(torch.tensor([[0., 0., 0., 0.], [0., 0., 0., 0.]]))
        self.layer.bias = nn.Parameter(torch.tensor([50., 0.]))
    
    def forward(self, x):
        return self.layer(x)
    
    def select_action(self, x, exploration=False):
        if exploration:
            raise ValueError('This model is not implemented for exploration')
        else:
            return self.forward(x)

In [ ]:
policy = Agent()

In [ ]:
gym.register('DoubleIntegrator', DoubleIntegratorEnv, max_episode_steps=100)

In [ ]:
class DoubleIntegrator(Env):
    def __init__(self):
        env = gym.make('DoubleIntegrator')
        
        self.is_discrete = False
        
        self.settings = {
            'noise': [],
            'max_frames': 999
        }
        
        self.h_function = nn.Sequential(
            FixedLinear(
                torch.tensor([
                    [1., 0, 0, 0],
                    [0, 1, 0, 0]
                ]),
                torch.tensor([0., 0.])
            ),
            Pow(2),
            FixedLinear(
                torch.tensor([
                    [-1 / 0.5 ** 2, 0],
                    [0, -1 / 0.5 ** 2]
                ]),
                torch.tensor([1., 1.])
            )
        )

        self.h_ids = [0, 1]
        self.std = [0.05**2/2, 0.05**2/2]
        self.env = IntegratorNoise(env)

In [ ]:
env = DoubleIntegrator()

In [ ]:
action = [50., 0.]


In [ ]:
cbf = CBF(env, nndm, policy,
          alpha=[0.9875, 0.9875],
          delta=[0., 0.],
          no_action_partitions=5,
          no_noise_partitions=5,
          stochastic=True)

In [ ]:
evaluator = Evaluator(env, cbf)
f, h = evaluator.mc_simulate(policy, 10, cbf_enabled=True, seed=[0., 0., 0., 0.])

In [ ]:
def plot_trajectories(num_agents, start_state, cbf_enabled=False):
    # plot the trajectories of the system, starting from the origin

    if start_state is None:
        start_state = [0., 0., 0., 0.]
    states = []

    for _ in tqdm(range(num_agents)):
        state_list = []
        state, _ = env.env.reset(seed=start_state)
        state = torch.tensor(state).unsqueeze(0)

        done = False

        while not done:   
            state_list.append(state.squeeze())

            # try cbf action - if cbf disabled or no safe actions available, just follow agent policy
            if cbf_enabled:
                try:
                    action = cbf.safe_action(state)
                except InfeasibilityError:
                    terminated = True
                    done = True
                    break
            else:
                action = policy.select_action(state, exploration=False)

            state, reward, terminated, truncated, _ = env.env.step(action.squeeze().detach().numpy())
            state = torch.tensor(state).unsqueeze(0)

            if torch.any(env.h_function(state.unsqueeze(0)) < 0).item():
                terminated = True

            done = terminated or truncated

        states.append(np.array(state_list))

    env.env.close()
    
    
    x_circle = np.linspace(-0.5, 0.5, 1001)
    
    y_circle_upper = [np.sqrt((0.5)**2 - x**2) for x in x_circle]
    y_circle_lower = [-np.sqrt((0.5)**2 - x**2) for x in x_circle]

    plt.plot(x_circle, y_circle_upper, color ="black")
    plt.plot(x_circle, y_circle_lower, color = "black")
    plt.fill_between(x_circle, y_circle_upper, y_circle_lower, color='g', alpha=0.3)
    x_axis = np.linspace(-0.5, 0.5, 3)
    y_axis = np.linspace(-0.5, 0.5, 3)

    plt.xticks(x_axis)
    plt.yticks(y_axis)

    plt.title("Trajectories")
    plt.xlabel("x")
    plt.ylabel("y")

    for trajectory in states:
        # https://stackoverflow.com/questions/21519203/plotting-a-list-of-x-y-coordinates 
        x, y, x_dot, y_dot = zip(*trajectory)
        plt.plot(x, y, c='b', alpha=0.1)


    plt.savefig("Double_integrator_trajectory_plot.png")
    plt.show()



In [ ]:
plot_trajectories(10, [0., 0., 0., 0.], cbf_enabled=True)